## Complexity of Rendering
- Geometry
- Shading
- Visibility

## Rendering
**Rendering** refers to the entire process that produces color values for pixels. 
- Compute what's visible. 
- Compute what color it is:
    - Determined by interaction of light and matter
    
### $\text{Rendering} = \text{Scene to image} = \text{Pinhole camera}$
![scene_to_image](images/scene_to_image.png)

### Pinhole camera
- Box with a tiny hole
- Inverted image
- Similar triangles
- **Perfect image if hole is infinitely small**
- Pure geometric optics
- **No depth of field issue**(everything in focus)
![pinhole](images/pinhole.jpg)

## Shading = What surfaces look like
- Surface/Scene properties:
    - Surface normal
    - Direction to light
    - Viewpoint
- Material Properties
    - Diffuse (matte)
    - Specular (shiny)
    - ...
- Light properties
    - Position
    - Intensity
    - ...
- Much more!
![shading_surface](images/shading_surface.png)

### Image synthesis is radiative transport
$$
\begin{aligned} \omega \cdot \nabla_{x} L(x, \omega)=& \epsilon(x, \omega) \\ &-\sigma_{t}(x) L(x, \omega) \\ &+\sigma_{s}(x) \int_{4 \pi} p\left(x, \omega, \omega^{\prime}\right) L\left(x, \omega^{\prime}\right) \mathrm{d} \omega^{\prime} \end{aligned}
$$

### Global Illumination
![illumination](images/illumination.png)

### Ray casting vs. Rasterization
![ray_casting_rasterizaion](images/ray_casting_rasterizaion.jpeg)

### Ray casting vs. Ray tracing
![ray_casting_tracing](images/ray_casting_tracing.png)

### Ray representation
- Two vectors: *Origin* and *Direction*
- Parametric line: $P(t) = \text{origin} + t \times \text{direction}$
    - In words, "Find smallest $t>0$ such that $P(t)$ lies on a surface in the scene."
    
![ray_representation](images/ray_representation.png)

### Ray tracing
- Advantages:
    - Generality: can render anythinig that can be intersected with a ray
    - **Easily allows recursion (shadows, reflections, etc.)**
- Disadvantages:
    - Harder to implement in hardware(less computation coherence, must fit entire scene in memory, worse memory behavior)

### Camera description
- Eye point $e$ (center)
- Orthobasis $u, v, w$ (horizontal, up, direction)

Different coordinate systems are used in a pipeline:
1. Object coordinates
2. World coordinates
3. View coordinates
4. Image coordinates
![view_frustum](images/view_frustum.jpeg)

### Image coordinates
Convenient to define "Normalized Image Coordinates" such that $x, y$ coordinates run from -1 to 1 regardless of the dimensions and aspect ratio of the image rectangle.

![nic](images/nic.jpeg)

### Ray generation in 2D
![ray_generation_2d](images/ray_generation_2d.jpeg)

### Ray generation in 3D
Similar to 2D. $y$ coordinate is treated just like $x$, except accounting for aspect ratio:
$$
\mathbf{r}=\mathrm{x} \cdot \mathbf{u}+\operatorname{aspect} \cdot \mathrm{y} \cdot \mathbf{v}+\mathrm{d} \cdot \mathbf{w}
$$

### Perspective vs. orthographic camera
Orthographic is parallel projection with neither foreshortening nor vanishing point.
![perspective_orthographic](images/perspective_orthographic.png)


Ray generation in orthographic:
- Origin = $\mathbf{e}+\mathrm{x} \cdot \mathrm{size} \cdot \mathbf{u}+\mathrm{y} \cdot \mathrm{size} \cdot \mathbf{v}$
- Direction is constant: $\mathbf{w}$

![orthographic](images/orthographic.png)

### 3D plane representation
- (Infinite) plane is defined by $\mathrm{P}_{\mathrm{o}}=\left(\mathrm{x}_{0}, \mathrm{y}_{0}, \mathrm{Z}_{0}\right)$ and $\mathrm{n}=(\mathrm{A}, \mathrm{B}, \mathrm{C})$.
- Implicit plane equation: 
$$
\begin{aligned} \mathrm{H}(\mathrm{P}) &=\mathrm{Ax}+\mathrm{By}+\mathrm{Cz}+\mathrm{D}=0 \\ &=\mathrm{n} \cdot \mathrm{P}+\mathrm{D}=0 \end{aligned}
$$

- Point-Plane distance?
    - If $n$ is normalized, distance to plane, $d = \mathrm{H}(\mathrm{P})$
    - $d$ is a **signed distance**!

![3d_plane_representation](images/3d_plane_representation.jpeg)


### Explicit vs. Implicit
- Ray equation is explicit: $\mathrm{P}(\mathrm{t})=\mathrm{R}_{\mathrm{o}}+\mathrm{t} \times \mathrm{R}_{\mathrm{d}}$
    - Parametric
    - Generates points
    - Hard to verify that a point is on the ray
    
- Plane equation is implicit: $\mathrm{H}(\mathrm{P})=\mathrm{n} \cdot \mathrm{P}+\mathrm{D}=0$
    - Solution of an equation
    - Does not generate points
    - Verifies that a point is on the plane

---------
## Intersections

### Intersection: Ray-Plane
Intersection means both equations are satisfied. Hence, insert explicit equation of ray into implicit equation of plane, and solve for $t$

$$
\begin{array}{l}{\mathrm{P}(\mathrm{t})=\mathrm{R}_{\mathrm{o}}+\mathrm{t} \times \mathrm{R}_{\mathrm{d}}} \\ {\mathrm{H}(\mathrm{P})=\mathrm{n} \cdot \mathrm{P}+\mathrm{D}=0} \\ {\mathrm{n} \cdot\left(\mathrm{R}_{\mathrm{o}}+\mathrm{t} \times \mathrm{R}_{\mathrm{d}}\right)+\mathrm{D}=0} \\ {\mathrm{t}=-\left(\mathrm{D}+\mathrm{n} \cdot \mathrm{R}_{\mathrm{o}}\right) / \mathrm{n} \cdot \mathrm{R}_{\mathrm{d}}}\end{array}
$$

![ray_plane_intersection](images/ray_plane_intersection.jpeg)

You can verify that,
- intersection is closer than previous: $\mathrm{P}(\mathrm{t})<\mathrm{t}_{\text { current }}$
- it is not out of range(behind eye): $\mathrm{P}(\mathrm{t})>\mathrm{t}_{\mathrm{min}}$

#### Normal 
Need surface normal for shading. (e.g. Diffuse: dot product between light direction and normal, clamp to zero). Also, normal is constant over a plane.


#### Math digression
- Duality points and planes are "dual" when you use homogenous coordinates
- Point $(x, y, z, 1)$
- Plane $(A, B, C, D)$
- Plane equation $\Rightarrow$ dot product
- You can map planes to points and points to planes in a dual space.
- Lots of cool equivalences
    - intersection of 3 planes define a point
    - 3 points define a plane!

### Intersection: Ray-Sphere 
- Implicit sphere equation: $\mathrm{H}(\mathrm{P})=\|\mathrm{P}\|^{2}-\mathrm{r}^{2}=\mathrm{P} \cdot \mathrm{P}-\mathrm{r}^{2}=0$
- Insert explicit equation of ray into implicit equation of sphere and solve for $t$: 
    - $\mathrm{P}(\mathrm{t})=\mathrm{R}_{\mathrm{o}}+\mathrm{t}^{*} \mathrm{R}_{\mathrm{d}} \quad ; \quad \mathrm{H}(\mathrm{P})=\mathrm{P} \cdot \mathrm{P}-\mathrm{r}^{2}=0$
    - $\left(\mathrm{R}_{\mathrm{o}}+\mathrm{tR}_{\mathrm{d}}\right) \cdot\left(\mathrm{R}_{\mathrm{o}}+\mathrm{tR}_{\mathrm{d}}\right)-\mathrm{r}^{2}=0$
    - $\mathrm{R}_{\mathrm{d}} \cdot \mathrm{R}_{\mathrm{d}} \mathrm{t}^{2}+2 \mathrm{R}_{\mathrm{d}} \cdot \mathrm{R}_{\mathrm{o}} \mathrm{t}+\mathrm{R}_{\mathrm{o}} \cdot \mathrm{R}_{\mathrm{o}}-\mathrm{r}^{2}=0$

![ray-sphere](images/ray-sphere.png)

- It's a quatratic equation: $a t^{2}+b t+c=0$

$$
\begin{array}{l}{\mathrm{a}=\mathrm{R}_{\mathrm{d}} \cdot \mathrm{R}_{\mathrm{d}}} \\ {\mathrm{b}=2 \mathrm{R}_{\mathrm{d}} \cdot \mathrm{R}_{\mathrm{o}}} \\ {\mathrm{c}=\mathrm{R}_{\mathrm{o}} \cdot \mathrm{R}_{\mathrm{o}}-\mathrm{r}^{2}}\end{array}
$$

- discriminant: $d=\sqrt{b^{2}-4 a c}$
- solutions: $t=\frac{-b \pm d}{2 a}$
    - Choose closest positive
    
- Sphere normal is simply $\mathrm{Q} /\|\mathrm{Q}\|$, where $\mathrm{Q}=\mathrm{P}(\mathrm{t})$ is an intersection point (for spheres centered at origin).


![sphere_normal](images/sphere_normal.png)

### Intersection: Ray-triangle  
You can either use
- ray-plane intersection followed by in-triangle test, or
- **barycentric  coordinates**

#### Barycentric definition of a plane
A non-degenerate triangle ($a,b,c$) defines a plane. Any point $\mathbf{P}$ on this plane can be written as $\mathbf{P}(\alpha, \beta, \gamma)=\alpha \mathbf{a}+\beta \mathbf{b}+\gamma \mathbf{c}$, where $\alpha+\beta+\gamma=1$

![barycentric1](images/barycentric1.png)

Hence, $\alpha, \beta, \gamma$ are barycentric coordinates. We can rewrite $\mathbf{P}$ by substituting $\alpha=1-\beta-\gamma$.

$$
\begin{aligned} \mathbf{P}(\alpha, \beta, \gamma) &=\alpha \mathbf{a}+\beta \mathbf{b}+\gamma \mathbf{c} \\ \mathbf{P}(\beta, \gamma) &=(1-\beta-\gamma) \mathbf{a}+\beta \mathbf{b}+\gamma \mathbf{c} \\ &=\mathbf{a}+\beta(\mathbf{b}-\mathbf{a})+\gamma(\mathbf{c}-\mathbf{a}) \end{aligned}
$$

So $\mathbf{P}$ is the **barycenter**, the point upon which the triangle would balance if weights of size $\alpha, \beta, \gamma$ are placed on points $\mathbf{a}, \mathbf{b}, \mathbf{c}$.

#### How to compute $\alpha, \beta, \gamma$?
They are the ratio of opposite sub-triangle area to total area:
$$\alpha=\mathrm{A}_{\mathrm{a}} / \mathrm{A} \quad \beta=\mathrm{A}_{\mathrm{b}} / \mathrm{A} \quad \gamma=\mathrm{A}_{\mathrm{c}} / \mathrm{A}$$

![barycentric2](images/barycentric2.png)

#### How to compute $\alpha, \beta, \gamma$? Use linear system!
Write $\mathbf{P}$ as a 2x2 linear system; $\mathbf{P}(\beta, \gamma)=\mathbf{a}+\beta \mathbf{e}_{1}+\gamma \mathbf{e}_{2}$, where $\mathbf{e}_{1}=(\mathbf{b}-\mathbf{a})$, $\mathbf{e}_{2}=(\mathbf{c}-\mathbf{a})$. 

So, $a+\beta e_{1}+\gamma e_{2}-P=0$ **should be zero**. Take inner products of this equation with $\mathbf{e}_{1}$ and $\mathbf{e}_{2}$.

$$
\begin{array}{l}{\left\langle e_{1}, a+\beta e_{1}+\gamma e_{2}-P\right\rangle= 0} \\ {\left\langle e_{2}, a+\beta e_{1}+\gamma e_{2}-P\right\rangle= 0}\end{array}
$$

which is equivalent to

$$
\left( \begin{array}{cc}{\left\langle\boldsymbol{e}_{1}, \boldsymbol{e}_{1}\right\rangle} & {\left\langle\boldsymbol{e}_{1}, \boldsymbol{e}_{2}\right\rangle} \\ {\left\langle\boldsymbol{e}_{2}, \boldsymbol{e}_{1}\right\rangle} & {\left\langle\boldsymbol{e}_{2}, \boldsymbol{e}_{2}\right\rangle}\end{array}\right) \left( \begin{array}{l}{\beta} \\ {\gamma}\end{array}\right)=\left( \begin{array}{c}{ c_{1}} \\ {c_{2}}\end{array}\right)
$$

where

$$
\left( \begin{array}{l}{c_{1}} \\ {c_{2}}\end{array}\right)=\left( \begin{array}{l}{\left\langle(\boldsymbol{P}-\boldsymbol{a}), \boldsymbol{e}_{1}\right\rangle} \\ {\left\langle(\boldsymbol{P}-\boldsymbol{a}), \boldsymbol{e}_{2}\right\rangle}\end{array}\right)
$$

and $\left\langle a, b\right\rangle$ is a dot product

#### Intersections with barycentric triangle
Set ray equation equal to barycentric equation:

$$
\begin{aligned} \mathbf{P}(\mathrm{t}) &=\mathbf{P}(\beta, \gamma) \\ \mathbf{R}_{\mathrm{o}}+\mathrm{t} * \mathbf{R}_{\mathrm{d}} &=\mathbf{a}+\beta(\mathbf{b}-\mathbf{a})+\gamma(\mathbf{c}-\mathbf{a}) \end{aligned}
$$

There is an intersection if $\beta+\gamma<1$ & $\beta>0$ & $\gamma>0$ and $t > t_{min} ...$

We have 3 variables with 3 equations:

$$
\begin{array}{l}{\mathrm{R}_{\mathrm{ox}}+\mathrm{tR}_{\mathrm{dx}}=\mathrm{a}_{\mathrm{x}}+\beta\left(\mathrm{b}_{\mathrm{x}}-\mathrm{a}_{\mathrm{x}}\right)+\gamma\left(\mathrm{c}_{\mathrm{x}}-\mathrm{a}_{\mathrm{x}}\right)} \\ {\mathrm{R}_{\mathrm{oy}}+\mathrm{tR}_{\mathrm{dy}}=\mathrm{a}_{\mathrm{y}}+\beta\left(\mathrm{b}_{\mathrm{y}}-\mathrm{a}_{\mathrm{y}}\right)+\gamma\left(\mathrm{c}_{\mathrm{y}}-\mathrm{a}_{\mathrm{y}}\right)} \\ {\mathrm{R}_{\mathrm{oz}}+\mathrm{tR}_{\mathrm{dz}}=\mathrm{a}_{\mathrm{z}}+\beta\left(\mathrm{b}_{\mathrm{z}}-\mathrm{a}_{\mathrm{z}}\right)+\gamma\left(\mathrm{c}_{\mathrm{z}}-\mathrm{a}_{\mathrm{z}}\right)}\end{array}
$$

Regroup and write in matrix form $\mathbf{A} \mathbf{x}=\mathbf{b}\left(=>\mathbf{x}=\mathbf{A}^{-1} \mathbf{b}\right)$:

$$
\left[ \begin{array}{lll}{a_{x}-b_{x}} & {a_{x}-c_{x}} & {R_{d x}} \\ {a_{y}-b_{y}} & {a_{y}-c_{y}} & {R_{d y}} \\ {a_{z}-b_{z}} & {a_{z}-c_{z}} & {R_{d z}}\end{array}\right] \left[ \begin{array}{c}{\beta} \\ {\gamma} \\ {t}\end{array}\right]=\left[ \begin{array}{c}{a_{x}-R_{o x}} \\ {a_{y}-R_{o y}} \\ {a_{z}-R_{o z}}\end{array}\right]
$$

We can solve the system by:

- inverting the matrix  by brute force (OK for 3x3), or
- use Cramer's rule

In the end, all triangle intersection algorithms have to perform these computations. Differences lie in what parts they precompute, and in which order they check for early-outs.

#### Solving $Ax = b$ wich Cramer's rule
Cramer's rule is used for solving one variable at a time in system of equations.

$$
\mathbf{\beta} = \frac{
\left| \begin{array}{lll}{a_{x}-R_{o x}} & {a_{x}-c_{x}} & {R_{d x}} \\ {a_{y}-R_{o y}} & {a_{y}-c_{y}} & {R_{d y}} \\ {a_{z}-R_{o z}} & {a_{z}-c_{z}} & {R_{d z}}\end{array}\right|
}{
|A|
}
$$

$$
\mathbf{\gamma} = \frac{
\left| \begin{array}{lll}{a_{x}-b_{x}} & {a_{x}-R_{o x}} & {R_{d x}} \\ {a_{y}-b_{y}} & {a_{y}-R_{o y}} & {R_{d y}} \\ {a_{z}-b_{z}} & {a_{z}-R_{o z}} & {R_{d z}}\end{array}\right|
}{
|A|
}
$$



$$
t = \frac{
\left| \begin{array}{lll}{a_{x}-b_{x}} & {a_{x}-c_{x}} & {a_{x}-R_{o x}} \\ {a_{y}-b_{y}} & {a_{y}-c_{y}} & {a_{y}-R_{o y}} \\ {a_{z}-b_{z}} & {a_{z}-c_{z}} & {a_{z}-R_{o z}}\end{array}\right|
}{
|A|
}
$$

**NOTE**: $| v |$ denotes the determinant.

#### Barycentric intersection Pros
- efficient
- stores no plane equation
- get the barycentric coordinates for free. 
    - useful for interpolation and texture mapping.

#### Barycentric interpolation
- Values $v_1, v_2, v_3$ defined at **a, b, c**; colors, normal, texture coordinates, etc.
- $\mathbf{P}(\alpha, \beta, \gamma)=\alpha \mathbf{a}+\beta \mathbf{b}+\gamma \mathbf{c}$ is the point
- $\mathrm{v}(\alpha, \beta, \gamma)=\alpha \mathrm{v}_{1}+\beta \mathrm{v}_{2}+\gamma \mathrm{v}_{3}$ is the barycentric interpolation of $v_1, v_2, v_3$ at point $\mathbf{P}$

Hence, once you know $\alpha, \beta, \gamma$, you can interpolate values using the same weights!

![barycentric3](images/barycentric3.png)

## Constructive Solid Geometry(CSG)
It's a neat way to build complex objects from simple parts using boolean operations. It's very easy when ray tracing but not so easy when not using ray tracing.

### How can we implement CSG?
![csg1](images/csg1.png)


![csg2](images/csg2.png)

In princible, CSG is simple but *floating point numbers are not exact*:
- e.g. points do not lie exactly on planes
- computing the intersection A vs. B is not necessarily the same as B vs. A
- line that results from intersecting two planes does not necessarily lie on either plane.

####
![](images/.png)

####
![](images/.png)

####
![](images/.png)

####
![](images/.png)

####
![](images/.png)



### 

### 

### 

### 